### Skolregistret Stockholms stadsarkiv
**Status**: started test webcrawl

* [Denna Notebook](https://github.com/perrohdin/Bildhistoria-Masterdata-och-Wikidata/blob/main/Notebook/Skolregistret%20Stockholms%20stadsarkiv.ipynb)

Databas copyright okänd, har besökt stadsarkivet och ombads skicka fråga med email
* [sok.stadsarkivet.stockholm.se/Databas/skolregistret](https://sok.stadsarkivet.stockholm.se/Databas/skolregistret/Sok) 1354 poster



In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2022-05-13 02:28:54.789452


In [2]:
import urllib3, re, time
import pandas as pd 
from bs4 import BeautifulSoup
from csv import DictReader, DictWriter
from tqdm import tqdm
req = urllib3.PoolManager()
    
urlbase = "https://sok.stadsarkivet.stockholm.se/Databas/skolregistret/Sok?sidindex="
skolKolumner = ["Url","Namn","Alt. namn","Byggår och arkitekt","Alternativt namn","Annexskolor",
           "Adress","Arkiv på Stadsarkivet","Rektorsområde","Arkiv hos övriga","Historik"]
dfTot = pd.DataFrame(columns=skolKolumner)
for i in tqdm(range(0,1300)):
    try:
        url = urlbase + str(i)
        res = req.request('GET', url)
        soup = BeautifulSoup(res.data, 'html.parser')
        contents = soup.find_all(class_= 'arende-table-row arende-table-row-alternating')
        for tablerow in contents:
            
            for rowLink in tablerow.find_all('a', href=True):
                schoolUrl = rowLink['href'].split("?sidindex")[0]
                namnUrl = rowLink.string
                #print("\t",schoolUrl,namnUrl)
             
            td_list = tablerow.find_all("td")
            #print(tablerow)
            namn = td_list[0].text.strip()
            altnamn = td_list[1].text.strip()
            adress = td_list[2].text.strip()
            byggår = td_list[3].text.strip()
            #print(namn,altnamn,adress,byggår)
                
            dfTot = dfTot.append({'Url': schoolUrl, 
                                  'Namn': namn, 
                                  'Alt. namn': altnamn,
                                  'Byggår och arkitekt':byggår,
                                  'Adress':adress
                                  }, ignore_index=True)
            #print(dfTot.shape)
    except Exception as e: 
        print(e)


100%|██████████| 1300/1300 [07:53<00:00,  2.75it/s]


In [3]:
dfTot.shape

(677, 11)

In [4]:
dfTot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677 entries, 0 to 676
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Url                    677 non-null    object
 1   Namn                   677 non-null    object
 2   Alt. namn              677 non-null    object
 3   Byggår och arkitekt    677 non-null    object
 4   Alternativt namn       0 non-null      object
 5   Annexskolor            0 non-null      object
 6   Adress                 677 non-null    object
 7   Arkiv på Stadsarkivet  0 non-null      object
 8   Rektorsområde          0 non-null      object
 9   Arkiv hos övriga       0 non-null      object
 10  Historik               0 non-null      object
dtypes: object(11)
memory usage: 58.3+ KB


In [5]:
dfTot

,Url,Namn,Alt. namn,Byggår och arkitekt,Alternativt namn,Annexskolor,Adress,Arkiv på Stadsarkivet,Rektorsområde,Arkiv hos övriga,Historik
0,Visa/larsboda-ro--1960-1991/7bb2e5c7-e0aa-4754...,Larsboda ro 1960-1991,,Huvudskola:\r\nLarsbodaskolan 1960-1991,NaN,NaN,,NaN,NaN,NaN,NaN
1,Visa/slattgardsskolan/51e42e66-35c7-4a49-9355-...,Slättgårdsskolan,,Grundskola åk 1-9 (före ht 2005 1-6)\r\n\r\nEg...,NaN,NaN,Frimurarvägen 11-17,NaN,NaN,NaN,NaN
2,Visa/rektorsomraden-ro--overlarardistrikt-od--...,Rektorsområden (Ro) / Överlärardistrikt (Öd) /...,,Stockholms folkskolor indelades 1903 i Överlär...,NaN,NaN,,NaN,NaN,NaN,NaN
3,Visa/tantoskolan/8aba544f-7f8c-4c08-98ca-013d1...,Tantoskolan,,Grundskola åk 1-3\r\n\r\nHyrda lokaler i hyres...,NaN,NaN,Flintbacken 32,NaN,NaN,NaN,NaN
4,Visa/stadens-allmanna-barnskola/44645a06-9851-...,Stadens allmänna barnskola,Allmänna barnskolan i Staden mellan broarna / ...,Öppnade 1821 som Stadens allmänna barnskola (A...,NaN,NaN,,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
672,Visa/engelbrekts-barnavards--och-husmodersskol...,Engelbrekts barnavårds- och husmodersskola,,"Privat yrkesskola, Kommunalt gymnasium\r\n\r\n...",NaN,NaN,först Odengatan 10 (privatvåning)\r\n1909-1988...,NaN,NaN,NaN,NaN
673,Visa/bromstensskolan/5aa2a36f-eb5f-4223-bee1-f...,Bromstensskolan,Bromstens folkskola,"Folkskola, Grundskola åk 1-6, Grundsärskola\r\...",NaN,NaN,Carl Barks väg 10,NaN,NaN,NaN,NaN
674,Visa/kunskapsskolan-tyreso/e2dbbd7c-a06a-47ce-...,Kunskapsskolan Tyresö,,Fristående grundskola åk 6-9.\r\n\r\nSkolan st...,NaN,NaN,Solkraftsvägen 15,NaN,NaN,NaN,NaN
675,Visa/olof-fryxells-flickskola/7c1a21e8-8f27-45...,Olof Fryxells flickskola,Bataljonspredikanten Olof Fryxells flickskola,Anges finnas 1848 på given adress i Staden mel...,NaN,NaN,Järntorget 80,NaN,NaN,NaN,NaN


In [7]:
dfTot.to_csv("SkolregistretStockholmsStadsarkiv.csv")